## 1. Project Overview & Objectives

In [22]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import re
import gc
import warnings

# Visualization settings
plt.style.use("seaborn-v0_8")
sns.set()
warnings.filterwarnings("ignore")

# Display settings
pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", lambda x: f"{x:,.4f}")


### 2. Load Data 

In [23]:
# Path to Mercari training data
DATA_PATH = "../data/raw/train.tsv"

# Load dataset
print("Loading data...")
df = pd.read_csv(DATA_PATH, sep="\t")

print("Loaded:")
df.info()


Loading data...
Loaded:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1482535 entries, 0 to 1482534
Data columns (total 8 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   train_id           1482535 non-null  int64  
 1   name               1482535 non-null  object 
 2   item_condition_id  1482535 non-null  int64  
 3   category_name      1476208 non-null  object 
 4   brand_name         849853 non-null   object 
 5   price              1482535 non-null  float64
 6   shipping           1482535 non-null  int64  
 7   item_description   1482529 non-null  object 
dtypes: float64(1), int64(3), object(4)
memory usage: 90.5+ MB


### 3. Data Cleaning

#### 3.1 Handling Missing Values

#### 3.2 Category Parsing & Splitting ( cat1/ cat2/ cat3 )

#### 3.3 Outlier Detection

#### 3.4 Data Type fixes

### 4. Category Structure Analysis

#### 4.1 Distribution of Category Levels

#### 4.2 Category-Level Price Statistics

#### 4.3 Category Popularity & Product Counts

### 5 Price Analysis

#### 5.1 Overall Price Distribution

#### 5.2 Price by Category

#### 5.3 Price Transformation Check (Log Price, etc.)

### 6 Product Attribute EDA

#### 6.1 Brand Analysis

#### 6.2 Ratings & Reviews

#### 6.3 Attribute Correlations

### 7 Preliminary Business Insights

### 8 Export Cleaned Dataset